In [ ]:
import numpy as np

# Configuration Parameters
PERIOD = 5.36  # cm - Structure period
MAGNETIC_FIELD = 0.16  # Tesla - External magnetic field
RESOLUTION = 16  # pixels/um - Simulation resolution
OUTPUT_DIR = 'output/dispersion/'

# Physical Constants
SPEED_OF_LIGHT = 3e8  # m/s
PLANCK_CONSTANT = 6.626e-34  # J·s
HBAR = PLANCK_CONSTANT / (2 * np.pi)
ELECTRON_CHARGE = 1.6e-19  # C
ELECTRON_MASS = 9.1e-31  # kg
G_FACTOR = 2  # Landé g-factor

# Material Properties
SATURATION_MAGNETIZATION = 200.5e3  # A/m
NORMALIZED_MAGNETIZATION = 0.178

# MEEP Simulation Parameters
ALPHA = 0.001  # Damping parameter
EPSILON_INF = 15  # Background permittivity

In [ ]:
import matplotlib.pyplot as plt
import meep as mp
import os
from datetime import datetime
from typing import Dict, Tuple, List

# Set matplotlib parameters for better plots
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12
%matplotlib inline

In [ ]:
def calculate_magneton_bohr() -> float:
    return (ELECTRON_CHARGE * HBAR) / (2 * ELECTRON_MASS)


def calculate_gyromagnetic_ratio(mu_b: float) -> Tuple[float, float]:
    gamma = (G_FACTOR * mu_b) / HBAR
    gamma_hz = gamma / (2 * np.pi)
    return gamma, gamma_hz


def calculate_yig_parameters(B0: float = MAGNETIC_FIELD, 
                           Ms: float = SATURATION_MAGNETIZATION,
                           ms: float = NORMALIZED_MAGNETIZATION) -> Dict[str, float]:
    # Calculate fundamental parameters
    mu_b = calculate_magneton_bohr()
    gamma, gamma_hz = calculate_gyromagnetic_ratio(mu_b)
    
    # Precession frequencies
    omega_pr = gamma * B0
    nu_pr = omega_pr / (2 * np.pi)
    
    # Wavelength and MEEP frequency
    lambda_pr = SPEED_OF_LIGHT / nu_pr
    f_meep = (1 / (lambda_pr * 100))*PERIOD  # Convert to MEEP units
    
    # MEEP parameters
    gamma_meep = gamma_hz
    
    # Sigma frequency
    omega_s = gamma * ms
    sigma = omega_s / (2 * np.pi)
    lambda_sigma = SPEED_OF_LIGHT / sigma
    f_sigma = (1 / (lambda_sigma * 100))*PERIOD
    
    # Calculate permeability for test frequency
    test_freq = 4.28e9 * 2 * np.pi
    denominator = omega_pr**2 - test_freq**2
    mu_x = 1 + (omega_pr * omega_s) / denominator
    k_x = (test_freq * omega_s) / denominator
    
    return {
        'mu_b': mu_b,
        'gamma': gamma,
        'gamma_hz': gamma_hz,
        'omega_pr': omega_pr,
        'nu_pr': nu_pr,
        'lambda_pr': lambda_pr,
        'f_meep': f_meep,
        'gamma_meep': gamma_meep,
        'sigma': sigma,
        'f_sigma': f_sigma,
        'omega_s': omega_s,
        'mu_x': mu_x,
        'k_x': k_x,
        'denominator': denominator
    }

In [ ]:
def print_parameters(params: Dict[str, float]) -> None:
    print("YIG Material Parameters:")
    print("=" * 50)
    print(f"Magnetic Induction (Tesla): {MAGNETIC_FIELD:.2f}")
    print(f"Magneton Bohr: {params['mu_b']:.3e}")
    print(f"Gyromagnetic Ratio (rad/Hz/T): {params['gamma']:.3e}")
    print(f"Gyromagnetic Ratio (Hz/T): {params['gamma_hz']:.3e}")
    print(f"Precession Frequency (rad/s): {params['omega_pr']:.3e}")
    print(f"Precession Frequency (Hz): {params['nu_pr']:.3e}")
    print(f"Wavelength (m): {params['lambda_pr']:.3f}")
    print(f"MEEP Frequency (cm): {params['f_meep']:.3f}")
    print(f"MEEP Gamma: {params['gamma_meep']:.3e}")
    print(f"Sigma: {params['sigma']:.3e}")
    print(f"MEEP Sigma Frequency (cm): {params['f_sigma']:.3f}")
    print(f"Mu(x): {params['mu_x']:.3f}")
    print(f"k(x): {params['k_x']:.3f}")

In [ ]:
# Calculate and display parameters
yig_params = calculate_yig_parameters()
print_parameters(yig_params)

## 4. Permeability Analysis and Visualization

In [ ]:
def calculate_permeability_spectrum(omega_pr: float, omega_s: float, 
                                  freq_range: Tuple[float, float] = (0, 2),
                                  num_points: int = 50) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    freq_min = freq_range[0] * omega_pr
    freq_max = freq_range[1] * omega_pr
    frequencies = np.linspace(freq_min, freq_max, num_points)
    
    mu_values = []
    k_values = []
    
    for freq in frequencies:
        denominator = omega_pr**2 - freq**2
        mu_x = 1 + (omega_pr * omega_s) / denominator
        k_x = (freq * omega_s) / denominator
        mu_values.append(mu_x)
        k_values.append(k_x)
    
    return frequencies, np.array(mu_values), np.array(k_values)

In [ ]:
def plot_permeability_components(frequencies: np.ndarray, mu_values: np.ndarray, 
                                k_values: np.ndarray, omega_pr: float) -> Tuple[plt.Figure, plt.Axes]:
    fig, ax = plt.subplots(figsize=(12, 8))
    
    # Plot permeability components
    ax.semilogx(frequencies, mu_values, 'b-', label='μ(ω)', linewidth=2)
    ax.semilogx(frequencies, k_values, 'r-', label='κ(ω)', linewidth=2)
    
    # Add reference lines
    ax.axvline(x=omega_pr, color='g', linestyle='--', label='ω_pr', linewidth=1.5)
    ax.axhline(y=0, color='k', linestyle=':', alpha=0.5)
    
    # Formatting
    ax.set_xlabel('Frequency ω (rad/s)', fontsize=14)
    ax.set_ylabel('Permeability Components', fontsize=14)
    ax.set_title('YIG Magnetic Permeability vs Frequency', fontsize=16)
    ax.legend(fontsize=12, loc='best')
    ax.grid(True, alpha=0.3, which='both')
    
    # Set reasonable y-limits
    y_min = min(np.min(mu_values), np.min(k_values))
    y_max = max(np.max(mu_values), np.max(k_values))
    ax.set_ylim(y_min * 1.1, y_max * 1.1)
    
    plt.tight_layout()
    return fig, ax

In [ ]:
# Calculate and plot permeability spectrum
frequencies, mu_vals, k_vals = calculate_permeability_spectrum(
    yig_params['omega_pr'], 
    yig_params['omega_s']
)

fig, ax = plot_permeability_components(frequencies, mu_vals, k_vals, yig_params['omega_pr'])
plt.show()

In [ ]:
def create_yig_material(f0: float, gamma: float, sigma: float, 
                       alpha: float = ALPHA, epsilon: float = EPSILON_INF) -> mp.Medium:
    susc = [mp.GyrotropicSaturatedSusceptibility(
        frequency=f0,
        gamma=gamma,
        sigma=sigma,
        alpha=alpha,
        bias=mp.Vector3(0, 0, 1)
    )]
    
    return mp.Medium(epsilon=epsilon, H_susceptibilities=susc)

In [ ]:
def create_photonic_crystal_geometry(material: mp.Medium, 
                                   N: int = 3, 
                                   r: float = 0.11) -> List[mp.Cylinder]:
    geometry = []
    for i in range(-N, N + 1):
        geometry.append(
            mp.Cylinder(
                r,
                material=material,
                center=mp.Vector3(0, i, 0)
            )
        )
    return geometry

In [ ]:
def create_photonic_crystal_geometry_single(material: mp.Medium, 
                                   N: int = 3, 
                                   r: float = 0.11) -> List[mp.Cylinder]:
    geometry = []
    geometry.append(
        mp.Cylinder(
            r,
            material=material,
            center=mp.Vector3(0, 0, 0)
        )
    )
    return geometry

In [ ]:
def setup_simulation(yig_params: Dict[str, float]) -> mp.Simulation:
    # Material parameters
    f0 = yig_params['f_meep']
    gamma = 2.8e-5
    sigma = yig_params['f_sigma']
    print(yig_params)
    
    # Create material
    yig_material = create_yig_material(f0, gamma, sigma)
    
    # Geometry
    geometry = create_photonic_crystal_geometry_single(yig_material)
    
    # Cell
    cell = mp.Vector3(1, 1)
    
    # Source
    fcen = 0.5
    df = 2
    src = [mp.Source(
        src=mp.GaussianSource(fcen, fwidth=df),
        component=mp.Ez,
        center=mp.Vector3(0.25, 0.15)
    )]
    
    # Create simulation
    sim = mp.Simulation(
        cell_size=cell,
        geometry=geometry,
        sources=src,
        resolution=RESOLUTION
    )
    
    return sim

In [ ]:
def run_dispersion_analysis(sim: mp.Simulation, k_interp: int = 19) -> np.ndarray:
    print("Running dispersion analysis...")
    Gamma = mp.Vector3(x=0,y=0,z=0)
    X = mp.Vector3(x=0.5,y=0,z=0)
    M =  mp.Vector3(x=0.5,y=0.5,z=0)
    k_points = mp.interpolate(k_interp, [Gamma, X, M, Gamma])
    # mp.Vector3(0), mp.Vector3(0.5), mp.Vector3(0.5, 0.5), mp.Vector3(0)
    freqs_result = sim.run_k_points(200, k_points)
    print("Dispersion analysis complete.")
    return freqs_result

In [ ]:
def plot_dispersion(freqs_result: np.ndarray, 
                   freq_cutoff: float = 0.8) -> Tuple[plt.Figure, plt.Axes]:
    fig, ax = plt.subplots(figsize=(10, 8))
    
    for i in range(len(freqs_result)):
        for freq in freqs_result[i]:
            if np.real(freq) > freq_cutoff:
                continue
            ax.scatter(i, np.real(freq), color='b', marker='.', s=20)
    
    ax.set_xlabel('k-point index', fontsize=14)
    ax.set_ylabel('Frequency (c/a)', fontsize=14)
    ax.set_title('Photonic Band Structure', fontsize=16)
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    return fig, ax

In [ ]:
def save_figure(fig: plt.Figure, filename: str, output_dir: str = OUTPUT_DIR) -> None:
    try:
        os.makedirs(output_dir, exist_ok=True)
        filepath = os.path.join(output_dir, filename)
        
        # Add timestamp to filename
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        base, ext = os.path.splitext(filename)
        filepath = os.path.join(output_dir, f"{base}_{timestamp}{ext}")
        
        fig.savefig(filepath, dpi=300, bbox_inches='tight')
        print(f"Figure saved to: {filepath}")
        
    except Exception as e:
        print(f"Error saving figure: {e}")

## 8. Main Execution

investigate harminv and H_susceptibilities

In [ ]:
# Run the complete analysis
print("Starting YIG dispersion analysis...\n")

# Set up simulation
sim = setup_simulation(yig_params)

# sim.k_point = mp.Vector3(0.5, 0.5)
# sim.run(mp.at_beginning(mp.output_epsilon), 
#         mp.at_every(1.2, mp.output_png(mp.Ez, "-C $EPS -Zc dkbluered")), 
#         until=100)

# Run dispersion analysis
freqs_result = run_dispersion_analysis(sim, 20)

# Plot results
fig_dispersion, ax_dispersion = plot_dispersion(freqs_result)
save_figure(fig_dispersion, "dispersion_diagram.png")

plt.show()